<img src="../figs/eh_logo.png" style="width: 200px;">

# EmptyHeaded SQL Query Language

This tutorial provides the basic information concerning our SQL query language
The EmptyHeaded SQL query language is standard SQL with a few differences for recursion.

1. [Joins](#join)
2. [Projections](#projection)
3. [Selections](#selection)
4. [Aggregations](#agg)
5. [Recursion](#recursion)

To run this tutorial you must have successfully completed the Getting Started Tutorial so that the database already exists on disk.

In [1]:
from emptyheaded import *
start()
eh = os.path.expandvars("$EMPTYHEADED_HOME")
db = Database.from_existing(eh+"/docs/notebooks/graph/db")

<a id='join'></a>
### Joins

A triangle query in SQL. Columns are named "a", "b", ...

In [5]:
db.eval("""
CREATE TABLE Triangle AS (
 SELECT e1.a, e2.a, e3.a 
 FROM Edge e1 
 JOIN Edge e2 ON e1.b = e2.a 
 JOIN Edge e3 ON e2.b = e3.a 
             AND e3.b = e1.a
)
""", useSql=True)

res = db.get("Triangle")
print(res.num_columns)
print(res.num_rows)
print(res.annotated)

3
9672060
False


<a id='projection'></a>
### Projections

In [12]:
db.eval("""
CREATE TABLE ProjectedTriangle AS (
  SELECT e1.a, e2.a 
  FROM Edge e1 
  JOIN Edge e2 ON e1.b = e2.a 
  JOIN Edge e3 ON e2.b = e3.a 
              AND e3.b = e1.a
)
""", useSql=True)

res = db.get("ProjectedTriangle")
print(res.num_columns)
print(res.num_rows)
print(res.annotated)

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



java.util.NoSuchElementExceptionPyRaisable: java.util.NoSuchElementException: key not found: b

<a id='selection'></a>
### Selections

In [8]:
db.eval("""
  CREATE TABLE FliqueSel AS (
  SELECT e1.a, e2.a, e3.a, e4.a 
  FROM Edge e1 
  JOIN Edge e2 ON e1.b = e2.a 
  JOIN Edge e3 ON e2.b = e3.a 
  JOIN Edge e4 ON e3.b = e4.a 
              AND e4.b = e1.a 
  JOIN Edge e5 ON e5.a = e1.a 
              AND e5.b = e2.b 
  JOIN Edge e6 ON e6.a = e1.b 
              AND e6.b = e3.b 
  JOIN Edge e7 ON e7.a = e1.a 
  WHERE e7.b = 0
)
""", useSql=True)

res = db.get("FliqueSel")
print(res.num_columns)
print(res.num_rows)
print(res.annotated)

4
3759972
False


<a id='agg'></a>
### Aggregations

The semantics of `COUNT(*)` should be the same as SQL. On the other hand, `SUM` is *not* the same as SQL, but has the same behavior as Datalog.

In [9]:
db.eval("""
CREATE TABLE BarbellAgg AS (
  SELECT COUNT(*) 
  FROM Edge e1 
  JOIN Edge e2 ON e1.b = e2.a 
  JOIN Edge e3 ON e2.b = e3.a 
              AND e3.b = e1.a 
  JOIN Edge e4 ON e4.a = e1.b 
  JOIN Edge e5 ON e5.a = e4.b 
  JOIN Edge e6 ON e5.b = e6.a 
  JOIN Edge e7 ON e6.b = e7.a 
              AND e7.b = e5.a
)
""", useSql=True)

res = db.get("BarbellAgg")
res.getDF()

,0
0,20371831447136


<a id='recursion'></a>

### Recursion

Recursion is defined using a base case and recursive case.

In [10]:
db.eval("""
CREATE TABLE N AS (
    SELECT SUM(e.a) 
    FROM Edge e
);

WITH RECURSIVE FOR 5 ITERATIONS PageRank AS (
  SELECT e.a, 1.0 / N FROM Edge e
  UNION
  SELECT e.a, 0.15+0.85*SUM(e.b) 
  FROM Edge e 
  JOIN PageRank pr ON e.b = pr.a 
  JOIN InvDegree i ON pr.a = i.a
)
""", useSql=True)

res = db.get("PageRank")
res.getDF()

,0,1
0,0,15.227066
1,1,0.529718
2,2,0.455440
3,3,0.512322
4,4,0.571060
5,5,0.498271
6,6,0.472287
7,7,0.601821
8,8,0.610678
9,9,1.121320


In [11]:
db.eval("""
WITH RECURSIVE SSSP AS (
  SELECT e.b, 1 
  FROM Edge e 
  WHERE e.a = 107
UNION
  SELECT e.b, 1 + MIN(e.a) 
  FROM Edge e 
  JOIN SSSP s ON s.a = e.a
)
""", useSql=True)

res = db.get("SSSP")
res.getDF()

,0,1
0,0,1
1,1,2
2,2,2
3,3,2
4,4,2
5,5,2
6,6,2
7,7,2
8,8,2
9,9,2


In [8]:
stop()